# Data Documentation Notes

The documentation is very thorough, and split by phase:
phase 1 = remote sensing only, phase 2 is core ground sampling, phase 3 has more tests

plots are roughly 1 acre

plot location is accurate within a mile, except <20% locations that were swapped with a similar location in the same county

#### [phase 2] Interesting attributes of:
- PLOT: lat/long/elev, water on plot, ecological subsection
- COND: forest type, stand age, stand size, productivity, stand origin, slope, aspect, physiographic class, disturbances, treatments, carbon content, stand structure, canopy cover pct, veg cover class
- SUBPLOT: slope, aspect, water depth, root disease severity
- SUBP_COND_CHNG_MTRX: condition, previous condition
- TREE: status, previous condition, species, diameter, height, class (sellability), crown class, year+cause of death(mortality), damage location/type/severity, volume, growth per year, crown position/light/vigor/density/dieback, foliage transparency, age, pct cull, trees per acre, carbon, damage cause
- TREE_GRM_COMPONENT: annual diameter/height/volume growth, status change
- SEEDLING: species, count, age, seedlings per acre 
- INVASIVE_SUBPLOT_SPP: species, cover pct
- P2VEG_SUBPLOT_SPP: species, growth habit, layer, cover pct
- DWM_COARSE_WOODY_DEBRIS: species, amount of decay, diameter/length/volume, dry mass, volume, logs per acre, charred?, volume per acre, mass per acre, carbon per acre, mortality per acre-year
- DWM_DUFF_LITTER_FUEL: duff/litter/fuel depth
- DWM_FINE_WOODY_DEBRIS: small/medium/large count/length
- DWM_MICROPLOT_FUEL: pct live/dead shrubs/herbs, pct litter cover, live/dead shrub/herb height
- DWM_RESIDUAL_PILE: shape, density, volume, biomass, carbon, piles per acre, amount decayed, species
- POP_ESTN_UNIT: land/total area
- POP_EVAL: name, description
- PLOTGEOM: lat/long, ecological subsection, watershed
- PLOTSNAP: water on plot, lat/long/elev
- REF_FOREST_TYPE: meaning, forest type group
- REF_FOREST_TYPE_GROUP: duff/litter/pile density/carbon-ratio
- REF_SPECIES: exists in _region_, soft or hardwood
- REF_PLANT_DICTIONARY: names, growth habit, lifespan, nativity

#### [phase 3] Interesting attributes of:
- OZONE_BIOSITE_SUMMARY: injured plant ratio, BIOSITE_INDEX (described as best summary of ozone injury), pct plants in severity classes
- OZONE_PLOT_SUMMARY: BIOSITE_INDEX, elevation, lat+long, plot size, aspect, terrain, soil depth+wetness
- LICHEN_PLOT_SUMMARY: summation, richness, evenness, diversity
- SOILS_EROSION: pct bare soil, pct compacted, compaction codes
- SOILS_SAMPLE_LOC: forest floor thickness, litter layer thickness
- SOILS_LAB: water content, density, coarse pct, (in)organic carbon pct, pct nitrogen, pH, sodium, potassium, magnesium, calcium, aluminum, manganese, iron, nickel, copper, zinc, cadmium, lead, sulfur, phosphorous
- VEG_PLOT_SPECIES: species code (join to REF_PLANT_DICTIONARY)
- VEG_SUBPLOT: land use, canopy/lichen/litter/mineral-soil/moss/road/rock/water/trash/wood cover pcts

## Interesting topics
- Species
- Mortality & injury rate (as proxy for health)
- Board feet lumber (as proxy for productivity)

I'll focus on species first...

## What to do with species data?

### Visualize the data
- Map: just plots, year, species, mortality (by cause), damage (by cause)
- single variable distributions
- plot aspect distribution radially

### Species prediction
This is a supervised learning problem, outputting class probability.
Target variable is TREE.SPCD, species code.
Can use REF_SPECIES to get species name from code.

Potential input variables:
- TREE.STATECD (if looking at more than one state)
- TREE.COUNTYCD
- TREE.DIA - diameter
- TREE.HT - height
- PLOT.LAT - latitude
- PLOT.LON- longitude
- PLOT.ELEV - elevation
- PLOT.WATERCD - nearby water codes, 7 options

These may not be enough for good predictions: will seek out more data if that's the case.

### Community detection
Try clustering algorithms

In [1]:
# setup
import pandas as pd

# show all columns & rows
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

let's inspect trees first, then plots and subplots

### Trees

In [2]:
# limited to 1 state to keep data size more manageable, may expand to whole country later
trees = pd.read_csv('data/WA_TREE.csv', index_col='CN')

# lowercase column labels to make it easier for me to type
trees.columns = [x.lower() for x in trees.columns]

# start simple
print('{} rows'.format(len(trees)))
trees.head()

/home/nick/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


338218 rows


,plt_cn,prev_tre_cn,invyr,statecd,unitcd,countycd,plot,subp,tree,condid,azimuth,dist,prevcond,statuscd,spcd,spgrpcd,dia,diahtcd,ht,htcd,actualht,treeclcd,cr,cclcd,treegrcd,agentcd,cull,damloc1,damtyp1,damsev1,damloc2,damtyp2,damsev2,decaycd,stocking,wdldstem,volcfnet,volcfgrs,volcsnet,volcsgrs,volbfnet,volbfgrs,volcfsnd,growcfgs,growbfsl,growcfal,mortcfgs,mortbfsl,mortcfal,remvcfgs,remvbfsl,remvcfal,diacheck,mortyr,salvcd,uncrcd,cposcd,clightcd,cvigorcd,cdencd,cdiebkcd,transcd,treehistcd,diacalc,bhage,totage,culldead,cullform,cullmstop,cullbf,cullcf,bfsnd,cfsnd,sawht,boleht,formcl,htcalc,hrdwd_clump_cd,sitree,created_by,created_date,created_in_instance,modified_by,modified_date,modified_in_instance,mortcd,htdmp,roughcull,mist_cl_cd,cull_fld,reconcilecd,prevdia,fgrowcfgs,fgrowbfsl,fgrowcfal,fmortcfgs,fmortbfsl,fmortcfal,fremvcfgs,fremvbfsl,fremvcfal,p2a_grm_flg,treeclcd_ners,treeclcd_srs,treeclcd_ncrs,treeclcd_rmrs,standing_dead_cd,prev_status_cd,prev_wdldstem,tpa_unadj,tpamort_unadj,tparemv_unadj,tpagrow_unadj,drybio_bole,drybio_top,drybio_stump,drybio_sapling,drybio_wdld_spp,drybio_bg,carbon_ag,carbon_bg,cycle,subcycle,bored_cd_pnwrs,damloc1_pnwrs,damloc2_pnwrs,diacheck_pnwrs,dmg_agent1_cd_pnwrs,dmg_agent2_cd_pnwrs,dmg_agent3_cd_pnwrs,mist_cl_cd_pnwrs,severity1_cd_pnwrs,severity1a_cd_pnwrs,severity1b_cd_pnwrs,severity2_cd_pnwrs,severity2a_cd_pnwrs,severity2b_cd_pnwrs,severity3_cd_pnwrs,unknown_damtyp1_pnwrs,unknown_damtyp2_pnwrs,prev_pntn_srs,disease_srs,dieback_severity_srs,damage_agent_cd1,damage_agent_cd2,damage_agent_cd3,centroid_dia,centroid_dia_ht,centroid_dia_ht_actual,upper_dia,upper_dia_ht,volcssnd,drybio_sawlog,damage_agent_cd1_srs,damage_agent_cd2_srs,damage_agent_cd3_srs,drybio_ag
CN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4871760,21864,NaN,2001,53,5,29,11,2,4871760,1,NaN,NaN,NaN,1,312,47,8.7,1.0,49.0,4.0,NaN,2.0,25.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6310,NaN,6.908000,6.908000,NaN,NaN,NaN,NaN,6.908000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-17,20004,NaN,2014-02-13,20004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.860000,NaN,NaN,NaN,210.356890,71.329310,12.352388,NaN,NaN,57.501926,147.019294,28.750963,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4871761,21864,NaN,2001,53,5,29,11,2,4871761,1,NaN,NaN,NaN,1,351,45,19.5,1.0,96.0,4.0,NaN,2.0,45.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4286,NaN,81.864998,82.640999,79.737000,80.492996,497.343994,520.109009,81.864998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-17,20004,NaN,2014-02-13,20004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.347000,NaN,NaN,NaN,2233.381933,455.121896,88.962737,NaN,NaN,530.524946,1388.733283,265.262473,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4879506,21864,NaN,2001,53,5,29,11,2,4879506,1,NaN,NaN,NaN,2,202,10,52.5,NaN,174.0,4.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,155.405308,661.612401,NaN,NaN,NaN,NaN,155.405308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-17,20004,NaN,2014-02-13,20004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.483999,NaN,NaN,NaN,5101.938856,794.474135,134.317912,NaN,NaN,1343.255409,3015.365452,671.627705,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [3]:
trees.isnull().sum()

plt_cn                         0
prev_tre_cn               252272
invyr                          0
statecd                        0
unitcd                         0
countycd                       0
plot                           0
subp                           0
tree                           0
condid                         0
azimuth                    13841
dist                       13869
prevcond                  232137
statuscd                       0
spcd                           0
spgrpcd                        0
dia                        16894
diahtcd                     1231
ht                         16894
htcd                       16689
actualht                   29493
treeclcd                   17661
cr                         63436
cclcd                      63436
treegrcd                  338218
agentcd                   318419
cull                       68415
damloc1                   245712
damtyp1                   336397
damsev1                   336397
damloc2   

In [4]:
trees.dia.describe()
# docs state diameter is in inches, these values seem reasonable. 
# some rows are missing this attribute

count    321324.000000
mean         12.827995
std          10.636604
min           1.000000
25%           6.200000
50%           9.100000
75%          15.100000
max         144.000000
Name: dia, dtype: float64

In [5]:
trees.ht.describe()
# measured in feet

count    321324.000000
mean         66.926741
std          38.925144
min           3.000000
25%          40.000000
50%          59.000000
75%          86.000000
max         280.000000
Name: ht, dtype: float64

In [6]:
trees.statecd.nunique() # just checking

1

In [7]:
print('{} counties'.format(trees.countycd.nunique()))
trees.countycd.value_counts()
# Washington has 39 counties, so a few are missing

35 counties


47    33662
59    27362
41    20699
7     20668
19    18490
9     18425
51    18365
61    15587
77    15541
57    15357
27    15194
73    14458
31    14333
33    13793
65    13543
37    13083
53     9700
15     6318
45     6240
49     6044
39     4684
63     2534
67     2365
13     2152
23     2073
35     1939
11     1725
69     1230
3       749
55      736
43      490
29      430
71      141
75      102
17        6
Name: countycd, dtype: int64

### Plots

In [8]:
plots = pd.read_csv('data/WA_PLOT.csv')

# lowercase column labels to make it easier for me to type
plots.columns = [x.lower() for x in plots.columns]

# start simple
print('{} rows'.format(len(plots)))
plots.head()

15715 rows


,cn,srv_cn,cty_cn,prev_plt_cn,invyr,statecd,unitcd,countycd,plot,plot_status_cd,plot_nonsample_reasn_cd,measyear,measmon,measday,remper,kindcd,designcd,rddistcd,watercd,lat,lon,elev,grow_typ_cd,mort_typ_cd,p2panel,p3panel,ecosubcd,congcd,manual,kindcd_nc,qa_status,created_by,created_date,created_in_instance,modified_by,modified_date,modified_in_instance,microplot_loc,declination,emap_hex,samp_method_cd,subp_examine_cd,macro_breakpoint_dia,intensity,cycle,subcycle,eco_unit_pnw,topo_position_pnw,nf_sampling_status_cd,nf_plot_status_cd,nf_plot_nonsample_reasn_cd,p2veg_sampling_status_cd,p2veg_sampling_level_detail_cd,invasive_sampling_status_cd,invasive_specimen_rule_cd,designcd_p2a,manual_db,subpanel
0,21860,153648002020004,64010497,NaN,2001,53,5,29,6,1,NaN,2001,12,1,NaN,0,558,NaN,NaN,48.304264,-122.702110,197.0,NaN,NaN,NaN,NaN,242Ad,5302.0,0.1,NaN,NaN,NaN,2012-12-13,20004,NaN,2017-02-06,489998,NaN,NaN,NaN,1,4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21863,153648002020004,64010497,NaN,2001,53,5,29,10,2,NaN,2001,10,26,NaN,0,558,NaN,NaN,48.256763,-122.696281,98.0,NaN,NaN,NaN,NaN,242Ad,5302.0,0.1,NaN,NaN,NaN,2012-12-13,20004,NaN,2017-02-06,489998,NaN,NaN,NaN,1,4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21864,153648002020004,64010497,NaN,2001,53,5,29,11,1,NaN,2000,8,1,NaN,0,558,NaN,NaN,48.189274,-122.127914,197.0,NaN,NaN,NaN,NaN,242Ab,5302.0,0.1,NaN,NaN,NaN,2012-12-13,20004,NaN,2017-02-06,489998,NaN,NaN,NaN,1,4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21865,153648002020004,64010497,NaN,2001,53,5,29,15,1,NaN,2000,8,3,NaN,0,558,NaN,NaN,48.189541,-122.762115,98.0,NaN,NaN,NaN,NaN,242Ad,5302.0,0.1,NaN,NaN,NaN,2012-12-13,20004,NaN,2017-02-06,489998,NaN,NaN,NaN,1,4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21866,153648002020004,64010497,NaN,2001,53,5,29,16,2,NaN,2001,10,26,NaN,0,558,NaN,NaN,48.237049,-122.695442,98.0,NaN,NaN,NaN,NaN,242Ad,5302.0,0.1,NaN,NaN,NaN,2012-12-13,20004,NaN,2017-02-06,489998,NaN,NaN,NaN,1,4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# how many null values in each column?
plots.isnull().sum()

cn                                    0
srv_cn                                0
cty_cn                                0
prev_plt_cn                       11925
invyr                                 0
statecd                               0
unitcd                                0
countycd                              0
plot                                  0
plot_status_cd                        0
plot_nonsample_reasn_cd           15074
measyear                              0
measmon                               0
measday                               0
remper                            11925
kindcd                                0
designcd                              0
rddistcd                           7025
watercd                            6447
lat                                 216
lon                                 216
elev                                216
grow_typ_cd                       11931
mort_typ_cd                       11931
p2panel                            1775


In [10]:
# what years are covered?
plots.invyr.value_counts().sort_index()

# first year has significantly more plots, otherwise close to 1000/yr

2001    1775
2002     991
2003     988
2004     990
2005     991
2006    1006
2007    1010
2008    1000
2009    1003
2010     998
2011     992
2012     996
2013     994
2014     993
2015     988
Name: invyr, dtype: int64

In [11]:
plots.lat.describe() # couple hundred missing 

count    15499.000000
mean        47.462728
std          0.912662
min         45.571160
25%         46.678621
50%         47.503538
75%         48.238934
max         49.001809
Name: lat, dtype: float64

In [12]:
plots.lon.describe()

count    15499.000000
mean      -120.656136
std          1.982464
min       -124.799169
25%       -122.145687
50%       -120.792705
75%       -118.987529
max       -116.945061
Name: lon, dtype: float64

In [13]:
plots.elev.describe()

count    15499.000000
mean      2305.268598
std       1717.271087
min          0.000000
25%        700.000000
50%       2100.000000
75%       3500.000000
max       9700.000000
Name: elev, dtype: float64

In [14]:
print('{} rows lack water code'.format(plots.watercd.isnull().sum()))
plots.watercd.value_counts().sort_index()
# most common value 0 means no water
# 8 is not in documentation... mysterious

6447 rows lack water code


0.0    6541
1.0    1342
2.0     125
3.0      27
4.0     998
5.0      36
8.0      68
9.0     131
Name: watercd, dtype: int64

### in the next notebook: visualizing some of this data